주소 컬럼의 빈값 채우기, 해시태그 컬럼의 공백 및 특수기호 등 제거 등 전처리  

In [1]:
import json, folium
import pandas as pd
import requests
from urllib.parse import quote

In [ ]:
#  데이터 컬럼명 변경하기
df.rename(columns ={'0':'관광지명','1':'주소','2':'연락처','3':'상세정보','4':'해시태그','5':'상세페이지_url'}, inplace=True)

In [ ]:
# 주소 컬럼이 null값인 행 확인하기
df[df['주소'].isnull()]

In [ ]:
# 주소가 빈값인 관광지명 리스트로 만들기
search_list = df[df['주소'].isnull()]['관광지명'].values.tolist()

In [ ]:
with open('도로명주소APIkey.txt') as file:
    road_key = file.read()

In [ ]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote ("렛츠런팜")}&resultType=json'
url = f'{base_url}?{params1}&{params2}'

In [ ]:
result = requests.get(url)
result.status_code

In [ ]:
json.loads(result.text)['results']['juso'][0]['roadAddr']

In [ ]:
#  null값인 주소 변경하기
df.loc[849,'주소'] = '제주특별자치도 제주시 조천읍 남조로 1649-48'
df.loc[883,'주소'] = '제주특별자치도 제주시 구좌읍 구좌해안로 237' 
df.loc[955,'주소'] = '제주 제주시 우도면 연평리 3'
df.loc[967,'주소'] = '제주특별자치도 서귀포시 안덕면 사계리 112-3'
df.loc[982,'주소'] = '제주 제주시 애월읍 평화로 1529'
df.loc[999,'주소'] = '제주 서귀포시 안덕면 상천리 산24'
df.loc[1014,'주소'] = '제주특별자치도 제주시 일도일동 1498'
df.loc[1073,'주소'] = '제주특별자치도 서귀포시 남원읍 신흥리 2083'
df.loc[1176,'주소'] = '제주특별자치도 서귀포시 안덕면 감산리 1803'
df.loc[1342,'주소'] = '제주특별자치도 제주시 추자면 영흥리'
df.loc[1362,'주소'] = '제주특별자치도 서귀포시 서귀동'

In [ ]:
#  해시태그_sub 열 생성 - 해시태그 열의 데이터값 공백 및 특수기호 등 제외
df['해시태그_sub'] = df['해시태그'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)

주소의 위도 경도 가져오기

In [2]:
with open('카카오APIkey.txt') as file:
    road_key = file.read()

In [3]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '제주특별자치도 서귀포시 성산읍 일출로 284-12'
url = f'{base_url}?query={quote(addr)}'
header = {'Authorization': f'KakaoAK {road_key}'}

In [4]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '제주특별자치도 서귀포시 성산읍 성산리 114',
    'b_code': '5013025921',
    'h_code': '5013025900',
    'main_address_no': '114',
    'mountain_yn': 'N',
    'region_1depth_name': '제주특별자치도',
    'region_2depth_name': '서귀포시',
    'region_3depth_h_name': '성산읍',
    'region_3depth_name': '성산읍 성산리',
    'sub_address_no': '',
    'x': '126.936800689963',
    'y': '33.462233758483'},
   'address_name': '제주특별자치도 서귀포시 성산읍 일출로 284-12',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '제주특별자치도 서귀포시 성산읍 일출로 284-12',
    'building_name': '',
    'main_building_no': '284',
    'region_1depth_name': '제주특별자치도',
    'region_2depth_name': '서귀포시',
    'region_3depth_name': '성산읍 성산리',
    'road_name': '일출로',
    'sub_building_no': '12',
    'underground_yn': 'N',
    'x': '126.936800689963',
    'y': '33.462233758483',
    'zone_no': '63643'},
   'x': '126.936800689963',
   'y': '33.462233758483'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count

In [5]:
result['documents'][0]['address']['region_3depth_h_name']

'성산읍'

In [25]:
result['documents'][0]['y']

'33.462233758483'

In [25]:
lng = result['documents'][0]['x']
lat = result['documents'][0]['y']

In [26]:
df_new = pd.read_csv('./관광지상제정보전처리_first.csv')

In [27]:
df_new.count()

관광지명    1486
주소      1486
연락처     1486
상세정보    1486
해시태그    1486
dtype: int64

In [ ]:
lat_list, lng_list = [],[]
for i in df_new.index:
    url = f'{base_url}?query={quote(df_new["주소"][i])}'
    result = requests.get(url, headers=header).json()
    try:
        lat_list.append(result['documents'][0]['y'])
        lng_list.append(result['documents'][0]['x'])
    except:
        print(df_new.관광지명[i])

In [183]:
#  잘못된 주소인 관광지 주소 변경하기
df_new[df_new['관광지명'] == '조일구락부 옛터']['주소'] 


1315    제주특별자치도 제주시 관덕로2길 11
Name: 주소, dtype: object

In [184]:
df_new.loc[1315,'주소'] = '제주특별자치도 제주시 관덕로2길 12-1'

In [185]:
df_new[df_new['관광지명'] == '조일구락부 옛터']['주소'] 

1315    제주특별자치도 제주시 관덕로2길 12-1
Name: 주소, dtype: object

In [ ]:
# 위도, 경도 가져오기
lat_list, lng_list = [],[]
for i in df_new.index:
    url = f'{base_url}?query={quote(df_new["주소"][i])}'
    result = requests.get(url, headers=header).json()
    try:
        lat_list.append(result['documents'][0]['y'])
        lng_list.append(result['documents'][0]['x'])
    except:
        print(df_new.관광지명[i])

In [ ]:
df_new['위도'] = lat_list
df_new['경도'] = lng_list

In [9]:
#  데이터 저장하기
df_new.to_csv('관광지상세정보위도경도_first.csv', index = False)
pd.read_csv('관광지상세정보위도경도_first.csv')

,관광지명,주소,연락처,상세정보,해시태그
0,성산일출봉(UNESCO 세계자연유산),제주특별자치도 서귀포시 성산읍 일출로 284-12,(+82) 064-783-0959,성산일출봉은 제주도의 다른 오름들과는 달리 마그마가 물속에서 분출하면서 만들어진 수...,"['#일출', '#오름', '#경관/포토', '#부모', '#자연경관', '#포토스..."
1,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,(+82) 064-728-1527,우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. ...,"['#섬속의섬', '#경관/포토', '#아이', '#맑음', '#자연경관', '#포..."
2,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,(+82) 064-900-8800,"사려니숲길은 제주의 숨은 비경 31곳 중 하나로, 비자림로를 시작으로 물찻오름과 사...","['#숲길', '#걷기/등산', '#친구', '#커플', '#흐림', '#봄', '..."
3,카멜리아힐,제주 서귀포시 안덕면 병악로 166,(+82) 064-800-6296,"카멜리아힐은 동양에서 가장 큰 동백 수목원으로 토종 동백부터 아기 동백, 유럽 동백...","['#경관/포토', '#커플', '#아이', '#맑음', '#겨울', '#힐링', ..."
4,협재해수욕장,제주특별자치도 제주시 한림읍 한림로 329-10,(+82) 064-728-3981,"반려동물의 경우, 해수욕장 개장 기간 내 지정 해수욕장 유영 구역의 입욕은 통제됩니...","['#일몰', '#해수욕장', '#액티비티', '#아이', '#맑음', '#여름',..."
...,...,...,...,...,...
1481,황우럭만화천국사회적협동조합,제주특별자치도 제주시 한림읍 내동남길 7-3,(+82) 064-796-7020,"황우럭만화천국사회적협동조합은 ""한림읍을 만화천국으로 만들자""는 비전을 가지고 황우럭...","['#친구', '#아이', '#체험관광', '#어린이', '#어트랙션']"
1482,효명사,제주특별자치도 서귀포시 남원읍 516로 815-41,(+82) 064-733-4950,한라산 중턱에 위치한 사찰 효명사\n큰 도로를 벗어나 우거진 숲속에 자리한 효명사 ...,"['#경관/포토', '#휴식/힐링', '#절', '#사찰', '#포토스팟', '#어..."
1483,후포해변,제주특별자치도 제주시 추자면 대서5길 87,(+82) 064-728-4292,파도에 부딪혀 동글동글해진 몽돌들이 있는 해변이다.,"['#해변', '#경관/포토', '#자연경관', '#포토스팟']"
1484,휘닉스 르쏠레이테라피,제주특별자치도 서귀포시 성산읍 섭지코지로 107 휘닉스제주아일랜드리조트 블루동 1층,(+82) 064-731-7709,"스파, 커플룸, 프라이빗룸으로 구성된 르쏠레이테라피센터는 성산 휘닉스호텔앤리조트 내...","['#스파', '#체험', '#호텔', '#어트랙션']"


In [30]:
df_l=pd.read_csv('관광지상세정보위도경도_first.csv')

In [2]:
df_tag= pd.read_csv('./관광지상세정보url_merge.csv')

In [3]:
df_tag.head(2)

,관광지명,주소,상세정보,해시태그_sub,평균별점,주소_동별,연락처,위도,경도,해시태그,상세페이지_url
0,성산일출봉(UNESCO세계자연유산),제주특별자치도 서귀포시 성산읍 일출로 284-12,성산일출봉은 제주도의 다른 오름들과는 달리 마그마가 물속에서 분출하면서 만들어진 수...,"일출,오름,경관,포토,부모,자연경관,포토스팟,유네스코,무장애관광,공영관광지",별점(5점만점에 5점),성산읍 성산리,(+82) 064-783-0959,33.462234,126.936801,"['#일출', '#오름', '#경관/포토', '#부모', '#자연경관', '#포토스...",https://www.visitjeju.net/kr/detail/view?conte...
1,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. ...,"섬속의섬,경관,포토,아이,맑음,자연경관,포토스팟,어린이,봄꽃,유채꽃,유네스코,무장애...",별점(5점만점에 5점),우도면 연평리,(+82) 064-728-1527,33.519472,126.951017,"['#섬속의섬', '#경관/포토', '#아이', '#맑음', '#자연경관', '#포...",https://www.visitjeju.net/kr/detail/view?conte...


해시태그 중 필요없는 단어가 포함된 행 삭제

In [13]:
tags_1 = ['의료관광']
list_a_1=df_tag[df_tag['해시태그_sub'].isin(tags_1)]

In [14]:
tags_2 = ['카페']
list_a_2=df_tag[df_tag['해시태그_sub'].isin(tags_2)]

In [20]:
(df_tag['해시태그_sub'].str.find('의료관광') >= 0).sum()

15

In [38]:
(df_tag['해시태그_sub'].str.find('펜션') >= 0).sum()

0

In [34]:
df_tag =df_tag[df_tag['해시태그_sub'].str.find('펜션') < 0]

In [45]:
df_tag[df_tag['관광지명'].str.find('펜션') >= 0]

,관광지명,주소,상세정보,해시태그_sub,평균별점,주소_동별,연락처,위도,경도,해시태그,상세페이지_url


In [49]:
pd.DataFrame(df_tag['관광지명'].unique()).to_excel('./test.xlsx')

In [51]:
df_tag.to_csv('./result/240122_tag_re.csv', index=False)

In [53]:
df=pd.read_csv('./result/240122_tag_re.csv')
len(df)

1230

In [41]:
df_tag[df_tag['관광지명'] == '카페']

,관광지명,주소,상세정보,해시태그_sub,평균별점,주소_동별,연락처,위도,경도,해시태그,상세페이지_url


In [15]:
df_last[df_last['관광지명'] == '곽지해수욕장']

,관광지명,주소,상세정보,해시태그_sub,평균별점,주소_동별,연락처,위도,경도,해시태그,상세페이지_url
28,곽지해수욕장,제주특별자치도 제주시 애월읍 애월원당길 (곽지리),"반려동물의 경우, 해수욕장 개장 기간 내 지정 해수욕장 유영 구역의 입욕은 통제됩니...","일몰,해수욕장,액티비티,아이,맑음,여름,자연경관,체험,레저,체험,해변,물놀이,어린이...",별점(5점만점에 5점),애월읍 곽지리,(+82) 064-728-3985,33.451916,126.322595,"['#일몰', '#해수욕장', '#액티비티', '#아이', '#맑음', '#여름',...",https://www.visitjeju.net/kr/detail/view?conte...


In [63]:
result['documents'][0]['address']

{'address_name': '제주특별자치도 제주시 애월읍 광령리 2873',
 'b_code': '5011025321',
 'h_code': '5011025300',
 'main_address_no': '2873',
 'mountain_yn': 'N',
 'region_1depth_name': '제주특별자치도',
 'region_2depth_name': '제주시',
 'region_3depth_h_name': '애월읍',
 'region_3depth_name': '애월읍 광령리',
 'sub_address_no': '',
 'x': '126.428787959221',
 'y': '33.4437798639377'}

In [71]:
#  관광지별 주소 동, 읍 만 가져오기
dong_list = []
for i in df_last.index:
    url = f'{base_url}?query={quote(df_last["주소"][i])}'
    result = requests.get(url, headers=header).json()
    try:
        dong_list.append(result['documents'][0]['address']['region_3depth_name'])
    except:
        dong_list.append('error')
        print(df_last.관광지명[i])

애월해안도로
곽지해수욕장
검멀레해변
하도해변
돈내코
문섬(서귀포해양도립공원)
연화못
섶섬(서귀포해양도립공원)
세화해안도로
영주산
별도봉
강정천유원지
구좌해안로
큰사슴이오름
민오름(봉개동)
대포포구
당케산물(당케포구)
산방산 수국
성산 유채꽃재배단지
거믄여해안경승지
굴동포구
남내소
덕천리 팔자 좋아 길
모라이
무수천
보롬이(하논, 화구구)
삼양방파제
새시물
샘이오름
솔동산8경
신양해안도로
신엄포구
알오름(성산읍)
예래해안도로
웃세오름(족은오름)
한남의숙터


In [72]:
dong_list

['성산읍 성산리',
 '우도면 연평리',
 '조천읍 교래리',
 '안덕면 상창리',
 '한림읍 협재리',
 '조천읍 함덕리',
 '구좌읍 월정리',
 '조천읍 교래리',
 '애월읍 봉성리',
 '성산읍 고성리',
 '안덕면 서광리',
 '해안동',
 '봉개동',
 '조천읍 대흘리',
 '구좌읍 종달리',
 '구좌읍 평대리',
 '서홍동',
 '성산읍 고성리',
 '구좌읍 김녕리',
 '우도면 연평리',
 '구좌읍 세화리',
 '성산읍 고성리',
 '대정읍 상모리',
 '하효동',
 '한림읍 금능리',
 '대정읍 가파리',
 'error',
 '안덕면 사계리',
 'error',
 '하원동',
 '동홍동',
 '봉개동',
 '용담이동',
 '안덕면 사계리',
 '구좌읍 세화리',
 '한림읍 협재리',
 '애월읍 하가리',
 '서홍동',
 '중문동',
 '안덕면 상천리',
 '색달동',
 '대정읍 가파리',
 '이호일동',
 '한림읍 금악리',
 '한림읍 협재리',
 '한경면 신창리',
 '구좌읍 김녕리',
 '남원읍 신례리',
 '중문동',
 'error',
 '애월읍 곽지리',
 '구좌읍 월정리',
 '서홍동',
 '애월읍 고내리',
 '표선면 성읍리',
 '대정읍 구억리',
 '표선면 성읍리',
 '삼도이동',
 '서홍동',
 '표선면 표선리',
 '구좌읍 평대리',
 '애월읍 광령리',
 '용담이동',
 '남원읍 위미리',
 '성산읍 삼달리',
 '표선면 가시리',
 '토평동',
 '색달동',
 '우도면 연평리',
 '조천읍 교래리',
 '',
 '월평동',
 '조천읍 선흘리',
 '용강동',
 '구좌읍 세화리',
 '조천읍 함덕리',
 '용담이동',
 '한경면 고산리',
 '하효동',
 '서귀동',
 '색달동',
 '한림읍 월령리',
 '구좌읍 송당리',
 '우도면 연평리',
 '조천읍 북촌리',
 '조천읍 선흘리',
 '표선면 표선리',
 '한경면 저지리',
 '표선면 가시리',
 '안덕면 서광리',
 '오라이동',
 '조천읍 교래리',
 'error

In [73]:
df_last['주소_동별'] = dong_list

In [184]:
#  잘못된 주소_동별인 변경하기
df_last[df_last['관광지명'] == '한남의숙터']['주소_동별'] 

1440    error
Name: 주소_동별, dtype: object

In [185]:
df_last.loc[1440,'주소_동별'] = '대정읍 보성리'

In [186]:
df_last[df_last['관광지명'] == '한남의숙터']['주소_동별'] 

1440    대정읍 보성리
Name: 주소_동별, dtype: object

In [190]:
df_last=df_last[['관광지명','주소','상세정보','해시태그_sub','평균별점','주소_동별','연락처','위도','경도','해시태그']]

In [ ]:
df_last

In [195]:
df_last.to_csv('./관광지정보최종_240111.csv',index = False)

In [3]:
jido_df=pd.read_csv('./관광지정보최종_240111.csv')

In [4]:
jido_df['해시태그_sub'].str.strip()

0       일출     오름     경관 포토     부모     자연경관     포토스팟  ...
1       섬속의섬     경관 포토     아이     맑음     자연경관     포토스팟...
2       숲길     걷기 등산     친구     커플     흐림     봄     자연...
3       경관 포토     커플     아이     맑음     겨울     힐링     자...
4       일몰     해수욕장     액티비티     아이     맑음     여름     ...
                              ...                        
1480                  친구     아이     체험관광     어린이     어트랙션
1481       경관 포토     휴식 힐링     절     사찰     포토스팟     어트랙션
1482                       해변     경관 포토     자연경관     포토스팟
1483                            스파     체험     호텔     어트랙션
1484    부모     친구     아이     커플     액티비티     야영장     캠...
Name: 해시태그_sub, Length: 1485, dtype: object

In [7]:
jido_df['해시태그_sub']=jido_df['해시태그_sub'].apply(lambda x: x.strip())

In [11]:
jido_df['해시태그_sub']=jido_df['해시태그_sub'].str.replace('  ', '')

In [21]:
jido_df['해시태그_sub']= jido_df['해시태그_sub'].str.replace(' ', ',')

In [22]:
jido_df['해시태그_sub']

0               일출,오름,경관,포토,부모,자연경관,포토스팟,유네스코,무장애관광,공영관광지
1       섬속의섬,경관,포토,아이,맑음,자연경관,포토스팟,어린이,봄꽃,유채꽃,유네스코,무장애...
2             숲길,걷기,등산,친구,커플,흐림,봄,자연경관,도보여행,도보,숲,단풍,공영관광지
3       경관,포토,커플,아이,맑음,겨울,힐링,자연경관,포토스팟,어린이,어트랙션,동백,수국,...
4       일몰,해수욕장,액티비티,아이,맑음,여름,자연경관,체험,레저,체험,해변,물놀이,어린이...
                              ...                        
1480                                  친구,아이,체험관광,어린이,어트랙션
1481                           경관,포토,휴식,힐링,절,사찰,포토스팟,어트랙션
1482                                   해변,경관,포토,자연경관,포토스팟
1483                                        스파,체험,호텔,어트랙션
1484    부모,친구,아이,커플,액티비티,야영장,캠핑장,체험,레저,체험,어린이,캠핑,언택트,어트랙션
Name: 해시태그_sub, Length: 1485, dtype: object

In [27]:
jido_df.to_csv('./관광지정보최종_240112.csv',index=False)